In [3]:
# librerias analytics
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

# librerias db
import boto3
import psycopg2
import pymysql
import configparser

# librerias para generar datos
import random
from faker import Faker
from datetime import datetime

# referencias de archivos
import constants as cons
import ddl

# CONN con RDS

In [17]:
config = configparser.ConfigParser()
config.read(cons.CONFIG_FILE)

['config.cfg']

In [18]:
rds_conn = boto3.client(
    'rds',
    aws_access_key_id=config.get(cons.USER, cons.ACCESS_KEY),
    aws_secret_access_key=config.get(cons.USER, cons.SECRET_KEY),
    region_name='us-east-1'
)

In [19]:
rds_instances_ids = []
aws_response = rds_conn.describe_db_instances()

for db_instance in aws_response['DBInstances']:
    db_instance_identifier = db_instance['DBInstanceIdentifier']
    db_engine = db_instance['Engine']
    if (db_engine == 'mysql'):
      aws_mysql = db_instance
    else:
      aws_postgres = db_instance
    print(f"La instancia '{db_instance_identifier}' utiliza el motor de base de datos '{db_engine}'")

La instancia 'ja3plqub98cr' utiliza el motor de base de datos 'mysql'
La instancia 'xux3br0jo1lp' utiliza el motor de base de datos 'postgres'


In [20]:
try:
    instance = rds_conn.describe_db_instances(
        DBInstanceIdentifier=config.get(cons.DB, cons.INSTANCE_ID)
    )
    RDS_HOSTNAME = instance.get('DBInstances')[0].get('Endpoint').get('Address')
    print(RDS_HOSTNAME)
except Exception as ex:
    print("Error!! ", ex)

xux3br0jo1lp.cr0suguo232c.us-east-1.rds.amazonaws.com


In [28]:
pg_conn = psycopg2.connect(
    dbname=config.get(cons.DB, cons.DB_NAME), 
    user=config.get(cons.DB, cons.DB_USERNAME),
    password=config.get(cons.DB, cons.DB_PASSWORD),
    port=int(config.get('DB', 'DB_PORT')),
    host=RDS_HOSTNAME
)

# Preguntas de Negocio

In [22]:
def get_data(conn, query):
    return pd.read_sql_query(query, conn)

In [29]:
def get_data(conn, query):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        col_names = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(result, columns=col_names)
        return df
    except Exception as e:
        print(f"Error: {e}")
        return pd.DataFrame()
    finally:
        cursor.close()

In [30]:
query_prueba = "SELECT * from dimFechas;"

In [31]:
datos = get_data(pg_conn, query_prueba)

In [32]:
datos

,idfecha,fecha_completa,dia_semana,num_dia_mes,dia_nombre,dia_abrev,semana_bandera,mes,mes_nombre,cuarto,anio,campo_timestamp


## 1. Articulos Mas Vendidos

### ¿Cuáles son los artículos más vendidos según distintos períodos de tiempos?

In [ ]:
best_selling_query = """
SELECT 
    anio AS year,
    mes_nombre AS month,
    nombre,
    SUM(cantidad) AS cantidad_total
FROM hechos h
JOIN dimArticulo dA ON h.idArticulo = dA.idarticulo
JOIN dimFechas dF ON h.idFecha = dF.idFecha
GROUP BY year, month, nombre
ORDER BY year, month, cantidad_total DESC
LIMIT 5;
"""
best_selling_df = get_data(pg_conn, best_selling_query)


worst_selling_query = """
SELECT 
    anio AS year,
    mes_nombre AS month,
    nombre,
    SUM(cantidad) AS cantidad_total
FROM hechos h
JOIN dimArticulo dA ON h.idArticulo = dA.idarticulo
JOIN dimFechas dF ON h.idFecha = dF.idFecha
GROUP BY year, month, nombre
ORDER BY year, month, cantidad_total ASC
LIMIT 5;
"""

worst_selling_df = get_data(pg_conn, worst_selling_query)

# Display the summary tables
print("Top 5 productos por mes")
print(best_selling_df)

print("\nPeores 5 productos por mes")
print(worst_selling_df)

In [ ]:
annual_sales_query = """
SELECT 
    anio AS year,
    dA.nombre,
    SUM(h.cantidad) AS total_quantity
FROM hechos h
JOIN dimArticulo dA ON h.idArticulo = dA.idarticulo
JOIN dimFechas dF ON h.idFecha = dF.idFecha
GROUP BY year, dA.nombre
ORDER BY year, total_quantity DESC;
"""

annual_sales_df = get_data(pg_conn, annual_sales_query)

# top 10
top_10_annual = annual_sales_df.groupby('year').apply(lambda x: x.nlargest(10, 'total_quantity')).reset_index(drop=True)

# plot
for year in top_10_annual['year'].unique():
    plt.figure(figsize=(12, 6))
    subset = top_10_annual[top_10_annual['year'] == year]
    plt.bar(subset['nombre'], subset['total_quantity'], color='blue')
    plt.title(f'Top 10 productos por año {year}')
    plt.xlabel('Articulo')
    plt.ylabel('Cantidad Vendida')
    plt.xticks(rotation=90)
    plt.show()

## 2. Clientes nuevos
### ¿Existe un cambio en que clientes nos compran, según distintos períodos??

In [ ]:
clients_query = """
SELECT 
    h.idPersona,
    dF.anio AS year,
    dF.mes_nombre AS month,
    h.fecha,
    h.cantidad,
    h.total
FROM hechos h
JOIN dimFechas dF ON h.idFecha = dF.idFecha
JOIN dimPersona dP ON h.idPersona = dP.idpersona
WHERE dP.tipo_persona = 'Client';
"""
clients_df = get_data(pg_conn, clients_query)

In [ ]:
clients_df['fecha'] = pd.to_datetime(clients_df['fecha'])

# fecha primera compra
clients_df['first_purchase'] = clients_df.groupby('idPersona')['fecha'].transform('min')
df_first_purchases = clients_df[clients_df['fecha'] == clients_df['first_purchase']]

# df de nuevos clientes
df_new_clients = df_first_purchases.sort_values(by=['idPersona', 'transaction_id']).drop_duplicates('idPersona')
df_new_clients['month_of_first_purchase'] = df_new_clients['first_purchase'].dt.month
df_new_clients['year_of_first_purchase'] = df_new_clients['first_purchase'].dt.year

# cantidad e ingresos por cliente
aggregated_data = clients_df.groupby(['idPersona', 'first_purchase']).agg(
    quantity_month=('cantidad', 'sum'),
    revenue_month=('total', 'sum')
).reset_index()

# agregar datos totales a df de clientes
df_new_clients = df_new_clients.merge(aggregated_data, on=['idPersona', 'first_purchase'], how='left')

# cantidad e ingresos por año
yearly_aggregates = clients_df.groupby(['idPersona', df['fecha'].dt.year]).agg(
    quantity_year=('cantidad', 'sum'),
    revenue_year=('total', 'sum')
).reset_index()

# agregar totales por año a df de clientes
df_new_clients = df_new_clients.merge(
    yearly_aggregates, 
    left_on=['idPersona', 'year_of_first_purchase'], 
    right_on=['idPersona', 'fecha'], 
    how='left'
)

# renombrar
df_new_clients = df_new_clients[[
    'idPersona', 'first_purchase', 'month_of_first_purchase', 
    'year_of_first_purchase', 'quantity_month', 'quantity_year', 
    'revenue_month', 'revenue_year'
]].drop_duplicates(subset=['idPersona'])  # unique 

# ver tabla
print(df_new_clients.head())

In [ ]:
# agrupar por mes para graficos
monthly_data = df_new_clients.groupby(['year_of_first_purchase', 'month_of_first_purchase']).agg(
    new_clients=pd.NamedAgg(column='idPersona', aggfunc='nunique'),  
    total_quantity=pd.NamedAgg(column='quantity_month', aggfunc='sum'),  
    total_revenue=pd.NamedAgg(column='revenue_month', aggfunc='sum')  
).reset_index()

monthly_data['date'] = pd.to_datetime(monthly_data[['year_of_first_purchase', 'month_of_first_purchase']].assign(day=1))

# sort por fecha para graficar
monthly_data = monthly_data.sort_values('date')

In [ ]:
# PLOTS
plt.figure(figsize=(12, 6))
plt.plot(monthly_data['date'], monthly_data['new_clients'], marker='o', linestyle='-', color='blue', label='Clientes Nuevos')
plt.title('Clientes nuevos por mes')
plt.xlabel('Mes, Año')
plt.ylabel('Número de clientes')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()

# Plotting Total Quantity
plt.figure(figsize=(12, 6))
plt.plot(monthly_data['date'], monthly_data['total_quantity'], marker='o', linestyle='-', color='green', label='Cantidad de artículos total')
plt.title('Cantidad de artículos comprados por clientes nuevos')
plt.xlabel('Mes, Año')
plt.ylabel('Cantidad total')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()

# Plotting Total Revenue
plt.figure(figsize=(12, 6))
plt.plot(monthly_data['date'], monthly_data['total_revenue'], marker='o', linestyle='-', color='red', label='Ingresos totales')
plt.title('Ingresos totales de clientes nuevos')
plt.xlabel('Mes, Año')
plt.ylabel('Ingresos totales')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()

## 3. Estacionalidad de Ventas
### ¿En qué periodo del año se vende más y en qué periodo se vende menos?

In [ ]:
seasonality_query = """
SELECT 
    EXTRACT(YEAR FROM dF.fecha_completa) AS year,
    EXTRACT(QUARTER FROM dF.fecha_completa) AS quarter,
    EXTRACT(MONTH FROM dF.fecha_completa) AS month,
    SUM(h.cantidad) AS total_quantity,
    SUM(h.total) AS total_revenue
FROM hechos h
JOIN dimFechas dF ON h.idFecha = dF.idFecha
GROUP BY year, quarter, month
ORDER BY year, quarter, month;
"""

In [ ]:
season_df = get_data(pg_conn, seasonality_query)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(seasonality_df['year'].astype(str) + '-' + seasonality_df['month'].astype(str), seasonality_df['total_quantity'], label='Cantidad Total', marker='o')
plt.plot(seasonality_df['year'].astype(str) + '-' + seasonality_df['month'].astype(str), seasonality_df['total_revenue'], label='Ingresos Total', marker='o')
plt.title('Estaacionallidad de ventas: Ingresos y Cantidad vendida')
plt.xlabel('Año-Mes')
plt.ylabel('Cantidad / Ingresos')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

## 4. Rentabilidad por producto
### ¿Cuál es el producto que genera más ganancia? ¿Y el que genera menos?

In [ ]:
profit_query = """
SELECT 
    dA.nombre,
    SUM(h.ganancia) AS total_profit
FROM hechos h
JOIN dimArticulo dA ON h.idArticulo = dA.idarticulo
GROUP BY dA.nombre
ORDER BY total_profit DESC
LIMIT 20;
"""
profit_df = get_data(pg_conn, profit_query)

In [ ]:
# Bar plot for Profit per Product
plt.figure(figsize=(12, 6))
plt.bar(profit_df['nombre'], profit_df['total_profit'], color='green')
plt.title('Rentabilidad por producto')
plt.xlabel('Producto')
plt.ylabel('Utilidad')
plt.xticks(rotation=90)
plt.show()

## 5. Rentabilidad por categorías
### Existe alguna relación entre la categoría y la rentabilidad?

In [ ]:
category_query = """
SELECT 
    dA.categoria,
    SUM(h.ganancia) AS total_profit,
    SUM(h.total) AS total_sales
FROM hechos h
JOIN dimArticulo dA ON h.idArticulo = dA.idarticulo
GROUP BY dA.categoria
ORDER BY total_profit DESC
LIMIT 10;
"""
category_df = get_data(pg_conn, category_query)

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 6))
ax2 = ax1.twinx()
category_df.plot(kind='bar', x='categoria', y='total_profit', ax=ax1, position=1, color='blue', label='Utilidad Total')
category_df.plot(kind='bar', x='categoria', y='total_sales', ax=ax2, position=0, color='green', label='Ingresos Total', alpha=0.6)
ax1.set_title('Utilidad y ventas por categoría')
ax1.set_xlabel('Categoría')
ax1.set_ylabel('Utilidad')
ax2.set_ylabel('Ventas')
ax1.set_xticklabels(category_df['categoria'], rotation=45)
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()